#### Import Libararies

In [2]:
!pip install datasets transformers scikit-learn torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.w

In [3]:
from datasets import load_dataset
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import numpy as np

#### Load Dataset

In [4]:
dataset = load_dataset("google-research-datasets/paws", "labeled_final")
train_dataset = dataset['train']
dev_dataset = dataset['validation']
test_dataset = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

#### Tokeniser and Model

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

#### Sentance Embedding

In [6]:
def get_bert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

#### Preprocess Data

In [ ]:
def preprocess_data(dataset):
    dataset = dataset.map(lambda x: {
        'sentence1_embedding': get_bert_embedding(x['sentence1']),
        'sentence2_embedding': get_bert_embedding(x['sentence2'])
    })
    return dataset

train_dataset = preprocess_data(train_dataset)
dev_dataset = preprocess_data(dev_dataset)
test_dataset = preprocess_data(test_dataset)

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

#### Combine Embeddings

In [ ]:
def combine_embeddings(example):
    sentence1_embedding = example['sentence1_embedding']
    sentence2_embedding = example['sentence2_embedding']
    combined_embedding = np.concatenate((sentence1_embedding, sentence2_embedding))
    return {'combined_embedding': combined_embedding}

train_dataset = train_dataset.map(combine_embeddings)
dev_dataset = dev_dataset.map(combine_embeddings)
test_dataset = test_dataset.map(combine_embeddings)

#### Training

In [ ]:
X_train = np.vstack(train_dataset['combined_embedding'])
y_train = np.array(train_dataset['label'])

X_dev = np.vstack(dev_dataset['combined_embedding'])
y_dev = np.array(dev_dataset['label'])

X_test = np.vstack(test_dataset['combined_embedding'])
y_test = np.array(test_dataset['label'])

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

#### Test and Validate

In [ ]:
y_pred_dev = model.predict(X_dev)
accuracy_dev = accuracy_score(y_dev, y_pred_dev)
precision_dev, recall_dev, f1_dev, _ = precision_recall_fscore_support(y_dev, y_pred_dev, average='binary')

print(f'Validation Accuracy: {accuracy_dev:.4f}')
print(f'Validation Precision: {precision_dev:.4f}')
print(f'Validation Recall: {recall_dev:.4f}')
print(f'Validation F1-Score: {f1_dev:.4f}')

In [ ]:
y_pred_test = model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')

print(f'Test Accuracy: {accuracy_test:.4f}')
print(f'Test Precision: {precision_test:.4f}')
print(f'Test Recall: {recall_test:.4f}')
print(f'Test F1-Score: {f1_test:.4f}')